In [ ]:
import os
from bs4 import BeautifulSoup
from time import sleep
from pymongo import MongoClient
from pprint import pprint
import concurrent.futures
import random
from copy import deepcopy
from datetime import datetime

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [2]:
#helper files
import get_proxies as proxies
from extractors import get_data 

#### Pull listing_url documents from mongoDB

In [ ]:
db = MongoClient().get_database('philadelphia_home_prices')

In [ ]:
col = db.get_collection('listing_urls')

In [ ]:
cursor = db.listing_urls.find({})
listings = list(cursor)

In [ ]:
#convert dictionary values to list
listing_urls_list = []
for i in listings:
    url = i['url']
    listing_urls_list.append(url)

In [ ]:
#check first 5 in list
listing_urls_list[0:5]

#### New collection for listing data

In [ ]:
db.list_collection_names()

In [ ]:
#uncomment the code you need to use
#db.drop_collection('listing_data')
#db.create_collection('listing_data')

In [ ]:
col = db.get_collection('listing_data')

In [ ]:
col.count_documents({})

#### Get free proxies from https://free-proxy-list.net/

In [3]:
#call function to get list of proxies
proxies_address = proxies.proxy_list();

2020-11-06 16:14:54,068 root   DEBUG    === Initialized Proxy Parsers ===
2020-11-06 16:14:54,069 root   DEBUG    	 FreeProxy parser of 'http://free-proxy-list.net' with required bandwidth: '150' KBs
2020-11-06 16:14:54,070 root   DEBUG    	 PremProxy parser of 'https://premproxy.com/list/' with required bandwidth: '150' KBs
2020-11-06 16:14:54,071 root   DEBUG    =================================
2020-11-06 16:14:54,590 root   DEBUG    Added 300 proxies from FreeProxy
2020-11-06 16:14:55,345 http_request_randomizer.requests.parsers.PremProxyParser DEBUG    Pages: {'', '08.htm', '12.htm', '13.htm', '06.htm', '14.htm', '10.htm', '09.htm', '04.htm', '05.htm', '02.htm', '07.htm', '11.htm', '03.htm'}
2020-11-06 16:14:55,906 http_request_randomizer.requests.parsers.js.UnPacker INFO     JS UnPacker init path: https://premproxy.com/js/148e0.js
2020-11-06 16:14:56,482 http_request_randomizer.requests.parsers.js.UnPacker DEBUG    portmap: {'ra613': '8080', 're79f': '32108', 'r1202': '47464', 'r

In [ ]:
len(proxies_address)

#### Pick which urls in list by index

In [ ]:
#might not be needed
scraped_urls = []
bad_proxies = []
slow_proxies = []

In [ ]:
#only run once
remaining_listing_urls_list = deepcopy(listing_urls_list)

In [ ]:
def scrape(urls):
    listing_data = {}
    #while len(remaining_listing_urls_list) > =:
    try:
        PROXY = random.choice(proxies_address)
        webdriver.DesiredCapabilities.CHROME['proxy']={"httpProxy":PROXY,
                                                       "ftpProxy":PROXY,
                                                       "sslProxy":PROXY,
                                                       "proxyType":"MANUAL",}

        driver = webdriver.Chrome(chromedriver)
        url = random.choice(remaining_listing_urls_list)
        t0 = time.time()
        driver.get(url)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        t1 = time.time()
        listing_data = get_data(soup, url)

        #will skip and move url to bad_urls if the page format is different than expected
        if t1-t0 > 180:
            slow_proxies.append(PROXY)
            proxies_address.remove(PROXY)
        if listing_data['Address'] != 'Not Found':
            remaining_listing_urls_list.remove(url)
            scraped_urls.append(url)
        if listing_data['Address'] == 'Not Found' or t1-t0<5:
            bad_proxies.append(PROXY)
            proxies_address.remove(PROXY)
    
    except:
        print('except ran, bad proxy')
        bad_proxies.append(PROXY)
        proxies_address.remove(PROXY)
        print(len(proxies_address))
    
    col.insert_one(listing_data)
    
    print('______________________________________________________')
    print(f'   Remaining urls: {len(remaining_listing_urls_list)}')
    print(f'     Scraped urls: {len(scraped_urls)}')
    print(f'Good proxies left: {len(proxies_address)}')
    print(f'      Bad proxies: {len(bad_proxies)}')
    print(f'     Slow proxies: {len(slow_proxies)}')
    
    driver.close()
    
    if len(proxies_address)<75:
        sleep(random.uniform(60,120))
    
    return listing_data

In [ ]:
def threaded_scrape(urls):
    threads = min(10, len(urls))
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        executor.map(scrape, urls)

In [ ]:
threaded_scrape(remaining_listing_urls_list)
#threaded_scrape('https://www.realtor.com/realestateandhomes-detail/2236-Napfle-St_Philadelphia_PA_19152_M35913-67774')

### Sandbox

In [ ]:
count = 0
while count < 20:
    listing_data = scrape(remaining_listing_urls_list)
    #print(count)
    #print(listing_data)
    count += 1

In [ ]:
"85.238.167.170:51915" in proxies_address

In [ ]:
col.count_documents({})

In [ ]:
'https://www.realtor.com/realestateandhomes-detail/4521-Tolbut-St_Philadelphia_PA_19136_M45152-02595' in remaining_listing_urls_list

In [ ]:
len(remaining_listing_urls_list)

In [ ]:
len(listing_urls_list)

In [4]:
import time

In [ ]:
t0 = datetime.now()
sleep(1)
t1 = datetime.now()
(t1-t0) 

In [ ]:
t0 = time.time()
sleep(1)
t1 = time.time()
t1-t0

In [ ]:
type(t0)

In [ ]:
time.time()

In [ ]:
driver

In [5]:
proxies_address[:10]

['87.247.19.126:58332',
 '46.41.135.121:8118',
 '191.101.39.203:80',
 '185.26.33.209:8080',
 '182.237.16.7:83',
 '82.81.169.142:80',
 '181.188.166.74:8080',
 '5.20.91.12:60792',
 '134.122.120.49:8118',
 '94.228.85.129:38510']